# Quantitative Momentum Strategy
We are going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as ptscore
import xlsxwriter

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## We are going to fetch one year price details of each stock

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2401326479169,
 'week52high': 147.87,
 'week52low': 89.32,
 'week52highSplitAdjustOnly': 150.86,
 'week52lowSplitAdjustOnly': 89.68,
 'week52change': 0,
 'sharesOutstanding': 16716105737,
 'float': 0,
 'avg10Volume': 63512106,
 'avg30Volume': 73359819,
 'day200MovingAvg': 132,
 'day50MovingAvg': 130.48,
 'employees': 150687,
 'ttmEPS': 4.47,
 'ttmDividendRate': 0.863403603058558,
 'dividendYield': 0.006203383393769787,
 'nextDividendDate': '',
 'exDividendDate': '2021-04-23',
 'nextEarningsDate': '2021-07-26',
 'peRatio': 30.809281050653194,
 'beta': 1.6055112626058687,
 'maxChangePercent': 53.58764643617855,
 'year5ChangePercent': 5.229848620198258,
 'year2ChangePercent': 1.8080829939004728,
 'year1ChangePercent': 0.5251405907391957,
 'ytdChangePercent': 0,
 'month6ChangePercent': 0.03921125957529686,
 'month3ChangePercent': 0.1219298199769208,
 'month1ChangePercent': 0.10846678973037902,
 'day30ChangePercent': 0.10901034963099679,
 'day5Chan

## Using the Batch API calls in order to reduce the time elapsed

In [4]:
def create_batch(x,n):
    for i in range(0,len(x),n):
        yield x[i:i+n]


In [5]:
batch_list = list(create_batch(stocks['Ticker'], 100))
batch_strings = []
for i in range(0, len(batch_list)):
    batch_strings.append(','.join(batch_list[i]))

df_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [6]:
df = pd.DataFrame(columns=df_columns)

for batch in batch_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={batch}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    
    # print(data)

    for stock in batch.split(','):
        df = df.append(
                        pd.Series(
                            [
                                stock, 
                                data[stock]['quote']['latestPrice'],
                                data[stock]['stats']['year1ChangePercent'],
                                'N/A'
                            ], index = df_columns
                        ), ignore_index = True
        )


## Sorting the data by top 50 returns in a year

In [7]:
df.sort_values('One-Year Price Return',ascending=False,inplace=True)
df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
275,LB,74.310,3.939463,N/A
441,TPR,43.300,2.313459,N/A
179,FCX,38.350,2.256766,N/A
148,DVN,29.870,1.871219,N/A
253,IVZ,26.670,1.791936,N/A
...,...,...,...,...
473,VRTX,206.280,-0.296628,N/A
118,CTL,11.000,None,N/A
165,ETFC,50.700,None,N/A
326,MYL,16.269,None,N/A


In [8]:
df = df[:51]
df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
275,LB,74.31,3.939463,N/A
441,TPR,43.30,2.313459,N/A
179,FCX,38.35,2.256766,N/A
148,DVN,29.87,1.871219,N/A
253,IVZ,26.67,1.791936,N/A
272,KSS,56.46,1.774809,N/A
410,SIVB,576.25,1.760597,N/A
208,GPS,34.40,1.7584,N/A
175,FANG,98.84,1.671439,N/A
314,MOS,31.95,1.654626,N/A


In [9]:
df.reset_index(drop=True,inplace=True)
df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,74.31,3.939463,N/A
1,TPR,43.30,2.313459,N/A
2,FCX,38.35,2.256766,N/A
3,DVN,29.87,1.871219,N/A
4,IVZ,26.67,1.791936,N/A
5,KSS,56.46,1.774809,N/A
6,SIVB,576.25,1.760597,N/A
7,GPS,34.40,1.7584,N/A
8,FANG,98.84,1.671439,N/A
9,MOS,31.95,1.654626,N/A


## Creating the Portfolio Size

In [10]:
def create_portfolio_size():
    global portfolio_size
    
    portfolio_size = input(' Please enter your portfolio amount : ')

    try:
        val = float(portfolio_size)
    except ValueError:
        portfolio_size = input("Please enter your portfolio amount : ")
    
    return portfolio_size

In [11]:
position_size = float(create_portfolio_size()) / len(df.index)

pd.set_option('mode.chained_assignment', 'warn')

for i in range(0, len(df['Ticker'])):

    total_share = position_size/df['Price'][i]
    df.loc[i,'Number of Shares to Buy'] = math.floor(total_share)


C:\Users\Preet\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [12]:
df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,74.31,3.939463,120
1,TPR,43.30,2.313459,206
2,FCX,38.35,2.256766,233
3,DVN,29.87,1.871219,299
4,IVZ,26.67,1.791936,335
5,KSS,56.46,1.774809,158
6,SIVB,576.25,1.760597,15
7,GPS,34.40,1.7584,260
8,FANG,98.84,1.671439,90
9,MOS,31.95,1.654626,280


## Building a more Realistic Strategy where we can differentiate between high quality momentum stock and low quality momentum stock.
We'll be focusing on : 
1. One Month price returns
2. Three Month price returns
3. Six Month price returns
4. One Year price returns

In [13]:
real_df_columns = [
    'Ticker',
    'Price',
    'One Year Price Return',
    'One Year Return Percentile',
    'Six Month Price Return',
    'Six Month Return Percentile',
    'Three Month Price Return',
    'Three Month Return Percentile',
    'One Month Price Return',
    'One Month Return Percentile',
    'Momentum Score',
    'Number of Shares to Buy'
]

real_df = pd.DataFrame(columns=real_df_columns)
real_df

,Ticker,Price,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,Momentum Score,Number of Shares to Buy


In [14]:
for batch in batch_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={batch}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    
    # print(data)

    for stock in batch.split(','):
        real_df = real_df.append(
                        pd.Series(
                            [
                                stock, 
                                data[stock]['quote']['latestPrice'],
                                data[stock]['stats']['year1ChangePercent'],
                                0,
                                data[stock]['stats']['month6ChangePercent'],
                                0,
                                data[stock]['stats']['month3ChangePercent'],
                                0,
                                data[stock]['stats']['month1ChangePercent'],
                                0,
                                0,
                                'N/A',
                            ], index = real_df_columns
                        ), ignore_index = True
        )

In [15]:
real_df

,Ticker,Price,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,Momentum Score,Number of Shares to Buy
0,A,150.90,0.712165,0,0.252219,0,0.16439,0,0.080947,0,0,N/A
1,AAL,22.54,0.682372,0,0.380874,0,-0.102813,0,-0.130566,0,0,N/A
2,AAP,218.53,0.500891,0,0.339284,0,0.133045,0,0.080982,0,0,N/A
3,AAPL,142.46,0.539744,0,0.038804,0,0.119697,0,0.107906,0,0,N/A
4,ABBV,118.87,0.211196,0,0.093122,0,0.066064,0,0.018761,0,0,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,119.11,0.379353,0,0.07982,0,0.063514,0,-0.032587,0,0,N/A
501,ZBH,167.35,0.380253,0,0.062354,0,0.021388,0,-0.017275,0,0,N/A
502,ZBRA,543.80,1.17222,0,0.388287,0,0.083285,0,0.060048,0,0,N/A
503,ZION,53.82,0.725386,0,0.263973,0,-0.023735,0,-0.076301,0,0,N/A


In [16]:
time_period = [
    'One Year',
    'Six Month',
    'Three Month',
    'One Month'
]


In [17]:
for i in real_df.index:
    for period in time_period:
        if real_df.loc[i, f'{period} Price Return'] == None:
            real_df.loc[i, f'{period} Price Return'] = 0

In [18]:

for i in real_df.index:
    for period in time_period:
        req_column = f'{period} Return Percentile'
        supp_column = f'{period} Price Return'
        
        # real_df.loc[i, req_column] = 0
        # print(float(real_df[supp_column][i]))
        # arr = real_df[supp_column]
        # arr_element = real_df[supp_column][i]
        
        real_df.loc[i, req_column] = ptscore(real_df[supp_column], real_df.loc[i, supp_column])/100


In [19]:
real_df

,Ticker,Price,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,Momentum Score,Number of Shares to Buy
0,A,150.90,0.712165,0.726733,0.252219,0.661386,0.16439,0.857426,0.080947,0.885149,0,N/A
1,AAL,22.54,0.682372,0.706931,0.380874,0.861386,-0.102813,0.033663,-0.130566,0.011881,0,N/A
2,AAP,218.53,0.500891,0.540594,0.339284,0.819802,0.133045,0.774257,0.080982,0.887129,0,N/A
3,AAPL,142.46,0.539744,0.568317,0.038804,0.182178,0.119697,0.740594,0.107906,0.928713,0,N/A
4,ABBV,118.87,0.211196,0.20396,0.093122,0.30495,0.066064,0.550495,0.018761,0.661386,0,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,119.11,0.379353,0.417822,0.07982,0.273267,0.063514,0.546535,-0.032587,0.283168,0,N/A
501,ZBH,167.35,0.380253,0.419802,0.062354,0.225743,0.021388,0.344554,-0.017275,0.40198,0,N/A
502,ZBRA,543.80,1.17222,0.924752,0.388287,0.875248,0.083285,0.605941,0.060048,0.829703,0,N/A
503,ZION,53.82,0.725386,0.740594,0.263973,0.689109,-0.023735,0.138614,-0.076301,0.10495,0,N/A


In [20]:
import statistics as stat

In [21]:
for i in real_df.index:
    momentum_percentiles = list()
    for period in time_period:
        momentum_percentiles.append(real_df[f'{period} Return Percentile'][i])
    real_df.loc[i,'Momentum Score'] = stat.mean(momentum_percentiles)

In [22]:
real_df

,Ticker,Price,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,Momentum Score,Number of Shares to Buy
0,A,150.90,0.712165,0.726733,0.252219,0.661386,0.16439,0.857426,0.080947,0.885149,0.782673,N/A
1,AAL,22.54,0.682372,0.706931,0.380874,0.861386,-0.102813,0.033663,-0.130566,0.011881,0.403465,N/A
2,AAP,218.53,0.500891,0.540594,0.339284,0.819802,0.133045,0.774257,0.080982,0.887129,0.755446,N/A
3,AAPL,142.46,0.539744,0.568317,0.038804,0.182178,0.119697,0.740594,0.107906,0.928713,0.60495,N/A
4,ABBV,118.87,0.211196,0.20396,0.093122,0.30495,0.066064,0.550495,0.018761,0.661386,0.430198,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,119.11,0.379353,0.417822,0.07982,0.273267,0.063514,0.546535,-0.032587,0.283168,0.380198,N/A
501,ZBH,167.35,0.380253,0.419802,0.062354,0.225743,0.021388,0.344554,-0.017275,0.40198,0.34802,N/A
502,ZBRA,543.80,1.17222,0.924752,0.388287,0.875248,0.083285,0.605941,0.060048,0.829703,0.808911,N/A
503,ZION,53.82,0.725386,0.740594,0.263973,0.689109,-0.023735,0.138614,-0.076301,0.10495,0.418317,N/A


In [23]:
top_50_real_df = real_df.sort_values('Momentum Score',ascending=False)

In [24]:
top_50_real_df

,Ticker,Price,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,Momentum Score,Number of Shares to Buy
175,FANG,102.81,1.670464,0.984158,1.103941,0.99802,0.23899,0.958416,0.155815,0.972277,0.978218,N/A
345,NVDA,836.32,1.175886,0.926733,0.570778,0.962376,0.476037,1.0,0.252882,0.99604,0.971287,N/A
149,DXC,42.36,1.561627,0.974257,0.586925,0.968317,0.314748,0.990099,0.062807,0.835644,0.942079,N/A
357,OXY,32.72,0.927523,0.855446,0.909951,0.992079,0.202615,0.922772,0.159024,0.974257,0.936139,N/A
251,IT,258.87,1.075225,0.910891,0.572252,0.964356,0.328463,0.99604,0.068954,0.847525,0.929703,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
184,FISV,110.02,0.103824,0.118812,-0.050675,0.045545,-0.115665,0.023762,-0.055751,0.154455,0.085644,N/A
188,FLT,272.10,0.00731,0.057426,-0.061436,0.041584,-0.083013,0.043564,-0.072818,0.112871,0.063861,N/A
290,LVS,54.78,0.186887,0.182178,-0.11112,0.019802,-0.141001,0.007921,-0.106616,0.039604,0.062376,N/A
112,CPB,47.19,-0.046647,0.027723,-0.041189,0.049505,-0.078627,0.047525,-0.065998,0.122772,0.061881,N/A


In [25]:
top_50_real_df = top_50_real_df[:51]
top_50_real_df.reset_index(drop=True,inplace=True)
top_50_real_df

,Ticker,Price,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,Momentum Score,Number of Shares to Buy
0,FANG,102.81,1.670464,0.984158,1.103941,0.99802,0.23899,0.958416,0.155815,0.972277,0.978218,N/A
1,NVDA,836.32,1.175886,0.926733,0.570778,0.962376,0.476037,1.0,0.252882,0.99604,0.971287,N/A
2,DXC,42.36,1.561627,0.974257,0.586925,0.968317,0.314748,0.990099,0.062807,0.835644,0.942079,N/A
3,OXY,32.72,0.927523,0.855446,0.909951,0.992079,0.202615,0.922772,0.159024,0.974257,0.936139,N/A
4,IT,258.87,1.075225,0.910891,0.572252,0.964356,0.328463,0.99604,0.068954,0.847525,0.929703,N/A
5,FTNT,251.21,0.752193,0.768317,0.658975,0.980198,0.311473,0.986139,0.133244,0.956436,0.922772,N/A
6,LB,76.99,3.988588,1.0,1.005033,0.99604,0.202159,0.920792,0.041763,0.770297,0.921782,N/A
7,WAT,359.10,0.957066,0.867327,0.436174,0.928713,0.241613,0.960396,0.105541,0.922772,0.919802,N/A
8,MRO,13.97,1.415811,0.968317,1.162828,1.0,0.20862,0.928713,0.030397,0.724752,0.905446,N/A
9,TGT,256.84,1.079262,0.912871,0.38778,0.873267,0.227133,0.946535,0.072763,0.869307,0.900495,N/A


In [26]:
position_size = float(create_portfolio_size())/ len(top_50_real_df.index)

In [27]:
for i in top_50_real_df.index:
    top_50_real_df.loc[i,'Number of Shares to Buy'] = math.floor(position_size/top_50_real_df.loc[i,'Price'])

C:\Users\Preet\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [28]:
top_50_real_df

,Ticker,Price,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,Momentum Score,Number of Shares to Buy
0,FANG,102.81,1.670464,0.984158,1.103941,0.99802,0.23899,0.958416,0.155815,0.972277,0.978218,786
1,NVDA,836.32,1.175886,0.926733,0.570778,0.962376,0.476037,1.0,0.252882,0.99604,0.971287,96
2,DXC,42.36,1.561627,0.974257,0.586925,0.968317,0.314748,0.990099,0.062807,0.835644,0.942079,1909
3,OXY,32.72,0.927523,0.855446,0.909951,0.992079,0.202615,0.922772,0.159024,0.974257,0.936139,2472
4,IT,258.87,1.075225,0.910891,0.572252,0.964356,0.328463,0.99604,0.068954,0.847525,0.929703,312
5,FTNT,251.21,0.752193,0.768317,0.658975,0.980198,0.311473,0.986139,0.133244,0.956436,0.922772,322
6,LB,76.99,3.988588,1.0,1.005033,0.99604,0.202159,0.920792,0.041763,0.770297,0.921782,1050
7,WAT,359.10,0.957066,0.867327,0.436174,0.928713,0.241613,0.960396,0.105541,0.922772,0.919802,225
8,MRO,13.97,1.415811,0.968317,1.162828,1.0,0.20862,0.928713,0.030397,0.724752,0.905446,5790
9,TGT,256.84,1.079262,0.912871,0.38778,0.873267,0.227133,0.946535,0.072763,0.869307,0.900495,314


In [29]:
xlsxwriter  = pd.ExcelWriter('momentum_strategy.xlsx',engine='xlsxwriter')
top_50_real_df.to_excel(xlsxwriter, sheet_name='Momentum Strategy', index = False)

In [30]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = xlsxwriter.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = xlsxwriter.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = xlsxwriter.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = xlsxwriter.book.add_format(
        {
            'num_format':'0.00%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [31]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['One Year Price Return', percent_template],
                    'D': ['One Year Return Percentile', percent_template],
                    'E': ['Six Month Price Return', percent_template],
                    'F': ['Six Month Return Percentile', percent_template],
                    'G': ['Three Month Price Return', percent_template],
                    'H': ['Three Month Return Percentile', percent_template],
                    'I': ['One Month Price Return', percent_template],
                    'J': ['One Month Return Percentile', percent_template],
                    'K': ['Momentum Score', percent_template],
                    'L': ['Number of Shares to Buy', integer_template]
                    }

for column in column_formats.keys():
    xlsxwriter.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    xlsxwriter.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [32]:
xlsxwriter.save()